In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost
from xgboost import XGBRegressor

In [2]:
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
df = pd.read_csv("C:/Users\monster\Desktop/NonLinearRegressionModels\RandomForest\Hitters.csv")
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(["Salary", "League", "Division", "NewLeague"], axis = 1).astype("float64")
X = pd.concat([X_, dms[["League_N", "Division_W", "NewLeague_N"]]], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

#Model Tuning#

In [4]:
xgb = XGBRegressor()

In [5]:
xgb.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'gamma': None,
 'gpu_id': None,
 'importance_type': 'gain',
 'interaction_constraints': None,
 'learning_rate': None,
 'max_delta_step': None,
 'max_depth': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [7]:
xgb_params = {"learning_rate": [0.1, 0.01, 0.5], "max_depth": [2,3,4,5,8], "n_estimators": [100,200,500,1000], 
              "colsample_bytree": [0.4, 0.7, 1] }

colsample_bytree -> olusturulan agaclarda, degiskenlerden alinacak olan alt kume orani 

In [8]:
xgb_cv_model = GridSearchCV(xgb, xgb_params, cv = 10, n_jobs = -1, verbose = 2).fit(X_train, y_train)

Fitting 10 folds for each of 180 candidates, totalling 1800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done 828 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done 1229 tasks      | elapsed:   24.1s
[Parallel(n_jobs=-1)]: Done 1674 tasks      | elapsed:   34.6s
[Parallel(n_jobs=-1)]: Done 1800 out of 1800 | elapsed:   36.4s finished


In [9]:
xgb_cv_model.best_params_

{'colsample_bytree': 0.4,
 'learning_rate': 0.1,
 'max_depth': 2,
 'n_estimators': 1000}

In [31]:
xgb_tuned = XGBRegressor(colsample_bytree = 0.7, learning_rate = 0.3, max_depth = 2, n_estimators = 1000).fit(X_train, y_train)

In [32]:
y_pred = xgb_tuned.predict(X_test)

In [33]:
np.sqrt(mean_squared_error(y_test, y_pred))

341.5665359031663